# Welcome to Tapy

<img src="pics/TaPy_logo.png"  width="250" height="250"/>

This notebook will illustrate the use of the TaPy library by going through a typical analysis

## Set up system

In [16]:
import os
import sys

Add TaPy to python path

In [31]:
root_folder = os.path.dirname(os.getcwd())
sys.path.append(root_folder)
import tapy
from tapy.grating_interferometer import GratingInterferometer
from tapy.roi import ROI

## Data Folders

Open Beam data path

In [20]:
path_ob = '../data/data_OB/'
assert os.path.exists(path_ob)

Sample data path

In [21]:
path_im = '../data/data_smp'
assert os.path.exists(path_im)

Dark Current data path

In [29]:
path_df = '../data/data_df'
assert os.path.exists(path_df)

## Loading Data 

In [30]:
o_grating = GratingInterferometer()
o_grating.load(folder=path_im)
o_grating.load(folder=path_ob, data_type='ob')
o_grating.load(folder=path_df, data_type='df')

## Normalization of the data 

We will use a normalization ROI.
```
 x0 = 3
 y0 = 5
 width = 20
 height = 40
```

In [33]:
norm_roi = ROI(x0=3, y0=5, width=20, height=40)
o_grating.normalization(roi=norm_roi)

True

## Calculate and display oscillation

In [34]:
osc_roi = ROI(x0=30, y0=1, width=1, height=1)

In [ ]:
o_grating.oscillation(roi=osc_roi)